# Unit 2: Follow a line

In this unit you will learn how to start using the most basic and also the most poverfull tool for perception in ROS: OpenCV. [**OpenCV**](https://docs.opencv.org/master/d9/df8/tutorial_root.html) is the most extensive and complet library for image recognition. With it you will be able to work with image like never before, applying filters, postprocessing, and working wiht images in any way you want.

## How to use OpenCV in ROS

As you might have gussed OpenCV is not a ROS library but its been integrated nicelz into with [**OpenCV_birdge**](http://wiki.ros.org/cv_bridge). This package allows the ROS imaging topics use the OpenCV image variable format.

For example, OpenCV images come in **BGR** image format while  regular ROS images are in the more standard RGB encoding. OpenCV_bridge provides a nice feature to convert between them. Also there are many other functions to transfer images to openCV variables transparently.

To learn how to use OpenCV you will use the RGB camera of this turtlebot.

<img src="Images/C5.png" width="450">

Note that this turtlebot is in a very strange environment. On the floor there is a Yellow path painted and some stars of different colours.

<img src="Images/C6.png" width="380">

What you will have to do is to make the robot move in the environment following the yellow line. For that we will divide the work in the following phases:
* Get images from ROS topic and convert into OpenCV format
* Process the Images using OpenCV libraries to obtain the data we want for the task
* Move the robot along the yellow line, based on the data obtained

Let's start

### 1. Get Images from a ROS topic and show them with OpenCV

Before doing anything, create a new package called **my_following_line_package** with dependency in *rospy*, and also a launch and scripts directory. 

The first step will be to get the images from a ROS topic, put them into the OpenCV format and show them in the Graphical Tool.

Here you have an example of how to do it:

<p style="color: white; background-color: green"> line_follower_basics.py </p>

In [ ]:
#!/usr/bin/env python

import roslib
import sys
import rospy
import cv2
import numpy as np
from cv_bridge import CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image

class LineFollower(object):
    
    def __init__(self):
        
        self.bridge_object = CvBridge()
        self.image_sub = rospy.Subscriber("/camera/rgb/image_raw", Image, self.camera_callback)
        
    def camera_callback(self,data):
        
        try:
            #we selec bgr8 because its the OpenCV encoding by default
            cv_image = self.bridge_object.imgmsg_to_cv2(data, desired_encoding = "bgr8")
        except CvBridgeError as e:
            print(e)
            
        cv2.imshow("Image window", cv_image)
        cv2.waitKey(1)
        
def main():
    line_follower_object = LineFollower()
    rospy.init_node('line_following_node', anonymous=True)
    try:
        rospy.spin()
    except KeyboardInterrupt:
        print("Shutting down")
    cv2.destoryAllWindows()
    
if __name__=='__main__':
    main()

<p style="color: white; background-color: green"> END line_follower_basics.py </p>

Here there are serval elements to comment on:

In [ ]:
import cv2
import numpy as np
from cv_bridge import CvBridgeError

This imports are the basic necessary to work with images in ROS.
<br>
You have OpenCVs library(cv2). Why the 2? Beacuase there is already a version 3.
<br>
You have the numpy library which make matrix and other operations easy to work with. CV_Bridge, which allows ROS work with OpenCV easily.

In [ ]:
self.image_sub = rospy.Subscriber("/camera/rgb/image_raw", Image, self.camera_callback)

This susbcriber to the image topic. This topic publishes information of type **sensor_msgs/image**. Execute the following command to see what all the different variables inside this message type:

The most important infomation here is:
* height and width: These are the dimensions in pixels of the camera. 
* encoding: How these pixels are encoded. This means what will each value in the data array mean. In this case its **rgb8**. This means that the values in data will be a color value represented as a red/green/blue in 8-bit integers.
* data: The Image data.
<br>

If you want the full docunmentation of this class, please reffer to: http://docs.ros.org/melodic/api/sensor_msgs/html/msg/Image.html

Thanks to the cv_bridge package, we can easily convert the image data contained in a ImageSensorData into a format that OpenCV understands. By converting it into OpenCV, we can use all the power of library to process the images of the robot.

In [ ]:
try:
    #we selec bgr8 because its the OpenCV encoding by default
    cv_image = self.bridge_object.imgmsg_to_cv2(data, desired_encoding = "bgr8")
except CvBridgeError as e:
    print(e)

Retrieve the image from a ROS topic and store it in an OpenCV variable. The var data is the one that contains a ROS message with the image captures by camera.

In [ ]:
cv2.imshow("Image window", cv_image)
cv2.waitKey(1)

This will open a gui where you can see the contents of the variable "cv_image". This is essential afterwards to see the effects fo the different filters and cropping of the image.

In [ ]:
cv2.destoryAllWindows()

This command will close all the image windows when the program is terminated.


When executing this program you will have to see the image by clicking on the **Graphical Tools** icon.

<p style="color: white; background-color: orange"> Exercise U2-1 </p>

So now you just have to create a python script with the command code and see if it works.
<br>
Create a package for the ocasion called "my_follow_line_package", and put that python file in a scripts folder.

<img src="Images/C7.png" width="250">

Then launch the code using the command below, and test that it actually works.

<p style="color: white; background-color: orange"> END  Exercise U2-1  </p>

<img src="Images/C8.png" width="380">


### 2. Apply Filter To the Image

The raw image is useless unless you filter it to only see the colour you want to track and you crop the parts os the image you are not interested in. This is to make your progarm faster.

We also need to extract some data from the images in a way that we can move the robot to follow the line.

#### First Step: Get Image Info and Crop the image

Before you start using the images for detecting things, you must take into account two things:
* One of the most basic data you need to work with images is the dimensions. Is it 800x600? 1200x1024? 60x60? This is crucial to position the elements detected in the image.
* And second is cropping the image. Its very important to work as soon as possible with the minimum size of image required for the task. This makes the detecting system much faster.

In [1]:
# We get image dimensions and crop the parts of the image we dont need
# Bare in mind that because its image matrix first value is start and second value is down limit.
# Select the limits so that it gets the line not to close, not to far and the minimum portion possible
# To make process faster.
height, width, channels = cv_image.shape
descentre = 160
rows_to_watch = 20
crop_img = cv_image[(height)/2+descentre:(height)/2+(descentre+rows_to_watch)][1:width]

So why this values and not other values? Well it depends on the task. In this case you are interested in lines that aren't too far away from the robot, nor to near. If you concertrate on the lines too far away it wont follow the lines, but it will just go across the map. On the other hand concentrating on lines too close won't give the robot time to adapt to chanches in the line.
<br>
Its also vital to ptimise the region of image result of croping. If its too big, too much data will be processed making your program too slow. On the other hand it has to have enough image to work with. 
<br>
At the end you will have to adapt it to each situation.
<br>
Do **Exercise U2.3** to test the effects of different values.

#### Second step: COnvert from BGR to HSV

Remmember that OpenCV works with BGR instead of RGB, for historical reasons (some cameras on the days when OpenCV was created worked with BGR).
<br>
Well it seems that it not very easy to work with neither RGB nor BGR to differenciate colours. Thats why HSV is used. The ides behind HSV is to remove the component of colour saturation. This way its easier to recognise the same color in different light conditions, which is a serious issue in image recognition. More infromation:https://en.wikipedia.org/wiki/HSL_and_HSV

<img src="Images/C9.png" width="380">

In [ ]:
#Convert from RGB to HSV
hsv = cv2.cvtColor(crop_img, cv2.COLOR_RGB2HSV)

# Define the Yellow Colour in HSV
#RGB
# [[[222,255,0]]]
#BGR
# [[[0,255,222]]]
'''
To know which color to track in HSV, Put in BGR. Use ColorZilla to get the color registered by the camera
>>> yellow = np.uint8([[[B, G, R]]])
>>> hsv_yellow = cv2.cvtColor(yellow, cv2.COLOR_BGR2HSV)
>>> print( hsv_yellow )
[[[[ 34 255 255]]]]
'''

lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([50, 255, 255])

So in this piece of code you are converting the croped_image(crop_img) into HSV.
<br>
Then you select which colour in HSV you want to track, that would be selecting from the base of the colour cone, a single point. Because HSV values are quite difficult to generate, its better that you use a colour picker tool like ***ColorZilla*** to pick the RGB coding of colour to track, in this case the yellow line in the simulation.

Once you havre it, use the example code given in python terminal or create a tiny program that uses numpy as np and cv2 to convert it to HSV. In the example given the colour of the line is HSV=[[[34 255 255]]].

Once you have it then you have to select an upper and lower bound to consider the region of the cone base that you will consider Yellow. Bigger the refion more gradients of your picked colour will be accepted. This will depend on how your robot detects and the colour variations in the image and how critical is mixing similar colours.

#### Third step: Apply the mask

Now you need to generate a version of the cropped image in  which you only see  two colours: black and white. The white will be all the colours you consider yellow and the rest will be black. Its a binary image.

Why do you need to do this? It has two functions:
* Doing this you dont have a continuous detection. It IS the Colour or its NOT, there is no in between.  This is vital for the centroid calculation that will be done after, because it only works on the principal of YES or NO.
* Second, it will allow afterwards generate the Result image in which you extract everything on the image except the colour line, seeing only what you are interested on seeing.

In [ ]:
# Threshold the HSV image to get only yellow colors
mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

In [ ]:
# Bitwise-AND mask and original image
res = cv2.bitwise_and(crop_img, crop_img, mask=mask)

You then merge the croped coloured image in HSV with the binary mask image, to colour only the detections, leaving the rest in black.

#### Fourth step: Get The Cetroids, draw a circle where the centroid is and show all  the images

Centroids is essence represent points in space in which mass concentrates, the center of mass. Centroid and centers of mass are the same thing as far as this course goes. And they are calculated using Integrals.
<br>
This is extrapolated into images. But instead of having mass, we have colour. The place where you have more of the colour you are looking for, then that's where the centroid will be. Its the center of the blobs seen in an image.
<br>
Thats why you applied the mask to make the image binary. This way you can calculate eassily where the center of mass is located. This is because its a describte function, not a continuos one. This means that it allows us to integrate in a discrete fasion and not need a function discrbing the fluctuations in cuantity of colour throughout the region.

These centrois are vital in blob tracking because they give you a precise point in space where the blob is. This you will use it to follow the blob and therefore follow the line.

This is needed to calculate the centroids of the colour blobs. You use the image moments for this:

In [1]:
# calculate centroid of the blob of binary image using ImageMoments
m = cv2.moments(mask, Flase)
try:
    cx, cy = m['m10']/m['m00'], m['m01']/m['m00']
except ZeroDivisionError:
    cx, cy = height/2, width/2

NameError: name 'cv2' is not defined

As you can see here, you will obtain the coordinates of the cropped image where the centroid of the positive colour yellow detection occur. If nothing is detected it will be positioned in the center of the image.



If you want a more detailed explaination an OpenCV exercises in all that can be obtained with contour features: https://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html

If you are interested in all the mathematical justification: https://en.wikipedia.org/wiki/Image_moment

In [ ]:
# Draw the centroid in the resultut image
# cv2.cricle(img, center, radius, color [, thickness[, lineType[, shift]]])
cv2.cricle(res, (int(cx), int(cy)), 10, (0, 0, 255), -1) #BGR

cv2.imshow("Original", cv_image)
cv2.imshow("HSV", hsv)
cv2.imshow("MASK", mask)
cv2.imshow("RES", res)

cv2.waitKey(1)

OpenCV allows you to draw a lot of things over the images, not only geometric shapes. But in this case a cricle will suffice.

In [ ]:
cv2.cricle(res, (centre_cricle_x, centre_cricle_y), LineWidth, (RGBColour of line), TypeOfLine)

We are using this feature to draw a cricle in the location of the calculated centroid.

<img src="Images/C10.png" width="380">

More Info: https://docs.opencv.org/3.0-beta/modules/imgproc/doc/drawing_functions.html

### 3. Move the TurtleBot based on the position of the Centroid

In [ ]:
error_x = cx - width/2;
twist_object = Twist()
twist_object.linear.x = 0.2;
twist_object.angular.z = -error_x / 100;
rospy.loginfo("ANGULAR VALUE SENT===>" + str(twist_object.angular.z))

# Make it start turning
self.movekobuki_object.move_robot(twist_object)

This control is pased on a Proportional control. This means that i will oscilate a lot and probably have an error. But is the simples way of moving the robot and has the job done.
<br>
It gives always a constant linear movement and the angular Z velocity depends on the difference between the centroid center in X and the center of the image.

To move the robot you can use this module to avoid some topic related problems when comunicating with Kobuki, like loosing first topics or similar issues:

<p style="color: white; background-color: green"> 
    move_robot.py 
</p>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist

class Move Kobuki(object):
    
    def __init__(self):
        self.cmd_vel_pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
        self.cmd_vel_subs = rospy.Subscriber('/cmd_vel', Twist, self.cmdvel_callback)
        self.last_cmdvel_command = Twist()
        self._cmdvel_pub_rate = rospy.Rate(10)
        self.shutdown_detected = False
        
    def cmdvel_callback(self, msg):
        self.last_cmdvel_command = msg
        
    def compare_twist_commands(self, twist1, twist2):
        LX = twist1.linear.x == twist2.linear.x
        LY = twist1.linear.y == twist2.linear.y
        LZ = twist1.linear.z == twist2.linear.z
        AX = twist1.angular.x == twist2.angular.x
        AY = twist1.angular.y == twist2.angular.y
        AZ = twist1.angular.z == twist2.angular.z
        equal = LX and LY and LZ and AX and AY and AZ
        if not equal:
            rospy.logwarn("The current Twist is not the same as the one sent, Resending")
        return equal
    
    def move_robt(self, twist_object):
        #we make this to avoid Topic loss, specially at the start
        current_equal_to_new =  False
        while (not(curent_equal_to_new) and not (self.shutdown_detected)):
            self.cmd_vel_pub.publish(twist_object)
            self._cmdvel_pub_rate.sleep()
            current_equal_to_new =  self.compare_twist_commands(twist1=self.last_cmdvel_command, 
                                                               twist2=twist_object)
            
    def clean_class(self):
        #stop Robot
        twist_object = Twist()
        twist_object.angular.z = 0.0
        self.move_robot(twist_object)
        self.shutdown_detected =  True
    
def main():
    rospy.init_node('move_robot_node', anonymous=True)
    
    movekobuki_object = MoveKobuki()
    twist_object =  Twist()
    #Make it start turning
    twist_object.angular.z = 0.5
    
    rate = rospy.Rate(5)
    
    ctrl_c = False
    def shutdown():
        #works better than the rospy.is_shut_down()
        movekobuki_object.clean_class()
        rospy.loginfo("shutdown time!")
        ctrl_c = True
    
    rospy.on_shutdown(shutdownhook)
    
    whiel not ctrl_c:
        movekobuki_object.move_robot(twist_object)
        rate.sleep()
        
if __main__ == '__main__':
    main()
    

<p style="color: white; background-color: green"> 
    END move_robot.py  
</p>

Here you have an example of how all this code would be put together:

<p style="color: white; background-color: green"> 
    follow_line_step_hsv.py 
</p>

In [ ]:
#!/usr/bin/env python
import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
from move_robot import MoveKobuki

class LineFollower(object):
    
    def __init__(self):
        
        self.bridge_object = CvBridge()
        self.image_sub = rospy.Subscriber('/camera/rgb/image_raw', Image, self.camera_callback)
        self.movekobuki_object = MoveKobuki()
        
    def camera_callback(self, data):
        
        try:
            #We celect rgb8 because its the OpenCV encoding by default
            cv_image = self.bridge_object.image_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)
            
        # We get image dimendions and crop the parts of the image we dont need
        # Bare in mind that because its image matrix first value is start and second value is down limit.
        # Select the limits so that  it gets the line not too close, not to far and the minimum portion possible
        # To make process faster
        height, width, channels = cv_image.shape
        
        #descentre = 160
        #row_to_watch = 20
        
        #descentre = 0
        #row_to_watch = 20
        
        descentre = 0
        row_to_watch = 200
        
        crop_img = cv_image[(height)/2 + descentre:(height)/2 + (descentre + rows_to_watch)][1:width]
        
        # Convert from RGB to HSV
        hsv = cv2.cvtColor(crop_img, cv2.COLOR_RGB2HSV)
        
        # Define the Yellow Colour in HSV
        #RGB
        #[[[222, 255, 0]]]
        #BGR
        #[[[0, 255, 222]]]
        """
        To know which color to track in HSV, Put in BGR. Use ColorZilla to get the color registered by the camera
        >>> yellow = np.unit8[[[B, G,R]]]
        >>> hsv_yellow = cv2.cvtColor(yellow, cv2.COLOR_BGR2HSV)
        >>> print(hsv_yellow)
        [[[34 255 255]]]        
        """
        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([50, 255, 255])
        
        lower_yellow = np.array([0, 55, 55])
        upper_yellow = np.array([255, 255, 255])
        
        lower_yellow = np.array([33, 254, 254])
        upper_yellow = np.array([36, 255, 255])
        
        #Blue Star, RGB = [3, 70, 255] --> HSV[111, 252, 225]
        # Create a Filter for Blue star
        lower_yellow = np.array([90, 249, 200])
        upper_yellow = np.array([131, 255, 255])
        
        #Threshold the HSV image to get only yellow colors
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        # Calculate centroid of the blob of binary image using ImageMoments
        m = cv2.moments(mask, False)
        try:
            cx,cy = m['m10']/m['m00'], m['m01']/m['m00']
        except ZeroDivisionError:
            cx, cy = height/2, width/2
            
        #Bitwise-And mask and original image
        res =  cv2.bitwise_and(crop_img, crop_img, mask = mask)
        
        # Draw the centroid in the resulted image
        # cv2.circle(img, center, radius, color[. thickness[, lineType[, shift]]])
        cv2.circle(res, (int(cx), int(cy)), 10, (0,0,255), -1)
        
        cv2.imshow("Original", cv_image)
        cv2.imshow("HSV", hsv)
        cv2.imshow("MASK", mask)
        cv2.imshow("RES", res)
        
        cv2.waitKey(1)
        
        error_x = cx - width / 2;
        twist_object = Twist()
        twist_object.linear.x = 0.2;
        twist_object.angular.z = -error_x / 100;
        rospy.loginfo("ANGULAR VALUE SENT===>" + str(twist_object.angular.z))
        # Make it start turning
        self.movekobuki_object.move_robot(twist_object)
        
    def clean_up(self):
        self.movekobuki_object.clean_class()
        cv2.destoryAllWindows()
        
def main():
    rospy.init_node('line_following_node', anonymous=True)
        
    line_follower_object = LineFollower()
        
    rate = rospy.Rate(5)
    ctrl_c = False
    def shutdownhook():
        #works better than the rospy.is_shut_down()
        line_follower_object.clean_up()
        rospy.loginfo("shutdown time!")
        ctrl_c =  True
            
    rospy.on_shutdown(shutdownhook)
        
    while not ctrl_c:
        rate.sleep()
            
if __name__=='__main__':
    main()


<p style="color: white; background-color: green"> 
    END  follow_line_step_hsv.py 
</p>

<p style="color: white; background-color: orange"> 
    Exercise U2-2 
</p>

Create a python script inside your package with the provided code. Execute it and see how it performs.
<br>
Try some improvements:
* Lower the speed of the robot to see if if works better. Chage the linear and the angular speed.
* Change the behaviour of the robot, maybe crate a recovery behaviour for when it looses the line.

<p style="color: white; background-color: orange"> 
    END Exercise U2-2 
</p>

<p style="color: white; background-color: orange"> 
    Exercise U2-3 
</p>

Try changing the colour to be tracked. Try to track back three different star colour. Use the colout picker for getting the exact RGB colour for:
* RedStar
* GreenStar
* BlueStar

<br>
Here you have an example of how it should look with the blue star.

<img src="Images/C11.png" width="380">

Also try changing the values of the upper and lower bounds to the following options and see results:
* Loose colour detection: lower_yellow = np.arrary([0, 50, 50]), upper_yellow = np.array([255, 255, 255])
* STRICT colour detection: lower_yellow = np.arrary([33, 254, 254]), upper_yellow = np.array([36, 255, 255])

<br>
Here you have an example of what you should see when changing the lower and upper:

<img src="Images/C12.png" width="600">

See that in the loose version all the colour green and yellow are detected. While in the strict, you can see that even in the yellow line, there is a part that is cut of because its slightly different for the camera sensor.

<p style="color: white; background-color: orange"> 
    END Exercise U2-3 
</p>

<p style="color: white; background-color: orange"> 
    Exercise U2-4
</p>

Try executing the code with different values in the **descentre** and the rows to watch and see how the follow line performs.<br> Try the following values:
* descrentre = 0, rows_to_watch = 20. This case is that you control the center of the image and a very small piece.
* descentre = 0, rows_to_watch = 200. This is the case in which you are controlling nearly all the lower part of the image.
* descentre =  200, rows_to_watch=20. This case you are controlling a small fraction of the lower part of the image only.

<br>
Test these values and see which one is better.

You should see images like these:

<img src="Images/C13.png" width="500">

<img src="Images/C14.png" width="500">

<img src="Images/C15.png" width="500">

<p style="color: white; background-color: orange"> 
    END Exercise U2-4 
</p>

From this first experience you might have seen that there are two main problems:
1. When the robot follows a single line, it works, but when various paths are avaliable it just turns crazy until only one is in sight. This is because you do not have the info of multiple blobs detection to then papply a policy to select one direction.
2. When the centroid is detected in the far end of the image the robot goes wild and turns too much and oscilates. This is due to the Proportional control used. This could be fixed using a complete PID control.

We will address these two issues.

### Additional Step: Follow Multiple Centroids

So this is the same code as before (*follow_line_step_hsv.py*) except for the fact that it tracks multiple blobs, letting you choose the path to follow.

In [ ]:
contours, _= cv2.findContout(mask, cv2.CHAIN_APPROX_TC89_L1)
rospy.loginfo("Number of centroids==>"+str(len(contours)))
centres = []
for i in range(len(contours[i]))
    moments = cv2.moments(contours[i])
    try:
        centres.append((int(moments['m10']/moments['m00']), int(moments['m01']/moments['m00'])))
        cv2.cricle(res, centres[-1], 10, (0, 255, 0), -1)
        
    except ZeroDividionError:
        pass

This uses the "findContours" functions to extract all the contours and then calculate the moments of each one. <br> If it the m00 is null, it considers that the centeres are useless and doesnt paint them. <br> It then paints a Green circle in each contour center. This gives the following result:

<img src="Images/C16.png" width="380">

Here you have the full code for refference:

<p style="color: white; background-color: green"> 
    follow_line_step_multiple.py 
</p>

In [ ]:
#!/usr/bin/env python
import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
from move_robot import MoveKobuki

class LineFollower(object):
    
    def __init__(self):
        
        self.bridge_object = CvBridge()
        self.image_sub = rospy.Subscriber('/camera/rgb/image_raw', Image, self.camera_callback)
        self.movekobuki_object = MoveKobuki()
        
    def camera_callback(self, data):
        
        try:
            #We celect rgb8 because its the OpenCV encoding by default
            cv_image = self.bridge_object.image_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)
            
        # We get image dimendions and crop the parts of the image we dont need
        # Bare in mind that because its image matrix first value is start and second value is down limit.
        # Select the limits so that  it gets the line not too close, not to far and the minimum portion possible
        # To make process faster
        height, width, channels = cv_image.shape
        
        #descentre = 160
        #row_to_watch = 20
        
        #descentre = 0
        #row_to_watch = 20
        
        descentre = 0
        row_to_watch = 200
        
        crop_img = cv_image[(height)/2 + descentre:(height)/2 + (descentre + rows_to_watch)][1:width]
        
        # Convert from RGB to HSV
        hsv = cv2.cvtColor(crop_img, cv2.COLOR_RGB2HSV)
        
        # Define the Yellow Colour in HSV
        #RGB
        #[[[222, 255, 0]]]
        #BGR
        #[[[0, 255, 222]]]
        """
        To know which color to track in HSV, Put in BGR. Use ColorZilla to get the color registered by the camera
        >>> yellow = np.unit8[[[B, G,R]]]
        >>> hsv_yellow = cv2.cvtColor(yellow, cv2.COLOR_BGR2HSV)
        >>> print(hsv_yellow)
        [[[34 255 255]]]        
        """
        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([50, 255, 255])
        
        lower_yellow = np.array([0, 55, 55])
        upper_yellow = np.array([255, 255, 255])
        
        lower_yellow = np.array([33, 254, 254])
        upper_yellow = np.array([36, 255, 255])
        
        #Blue Star, RGB = [3, 70, 255] --> HSV[111, 252, 225]
        # Create a Filter for Blue star
        lower_yellow = np.array([90, 249, 200])
        upper_yellow = np.array([131, 255, 255])
        
        #Threshold the HSV image to get only yellow colors
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        # Calculate centroid of the blob of binary image using ImageMoments
        m = cv2.moments(mask, False)
        try:
            cx,cy = m['m10']/m['m00'], m['m01']/m['m00']
        except ZeroDivisionError:
            cx, cy = height/2, width/2
            
        #Bitwise-And mask and original image
        res =  cv2.bitwise_and(crop_img, crop_img, mask = mask)
        
        contours,_= cv2.findColours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1)
        rospy.loginfo("Number of centroids==>"+str(len(contours)))
        centres = []
        for i in range(len(contours)):
            moments = cv2.moments(contours[i])
            try:
                centres.append((int(moments['m10']/moments['m00']), int(moments['m01']/moments['m00'])))
                cv2.circle(res, centres[-1], 10, (0, 255, 0), -1)
            except ZeroDivisionError:
                pass
            
        rospy.loginfo(str(centres))
                               
        
        # Draw the centroid in the resulted image
        # cv2.circle(img, center, radius, color[. thickness[, lineType[, shift]]])
        cv2.circle(res, (int(cx), int(cy)), 10, (0,0,255), -1)
        
        cv2.imshow("Original", cv_image)
        #cv2.imshow("HSV", hsv)
        #cv2.imshow("MASK", mask)
        cv2.imshow("RES", res)
        
        cv2.waitKey(1)
        
        error_x = cx - width / 2;
        twist_object = Twist()
        twist_object.linear.x = 0.2;
        twist_object.angular.z = -error_x / 100;
        rospy.loginfo("ANGULAR VALUE SENT===>" + str(twist_object.angular.z))
        # Make it start turning
        self.movekobuki_object.move_robot(twist_object)
        
    def clean_up(self):
        self.movekobuki_object.clean_class()
        cv2.destoryAllWindows()
        
def main():
    rospy.init_node('line_following_node', anonymous=True)
        
    line_follower_object = LineFollower()
        
    rate = rospy.Rate(5)
    ctrl_c = False
    def shutdownhook():
        #works better than the rospy.is_shut_down()
        line_follower_object.clean_up()
        rospy.loginfo("shutdown time!")
        ctrl_c =  True
            
    rospy.on_shutdown(shutdownhook)
        
    while not ctrl_c:
        rate.sleep()
            
if __name__=='__main__':
    main()

<p style="color: white; background-color: green"> 
    End follow_line_step_multiple.py 
</p>

<p style="color: white; background-color: orange"> 
    Exercise U2-5
</p>

Create a new python with this multiple contours system and try the following
* Make the robot when faced with a dilema, select the right center to follow so that it stays in the cicular loop path.

<p style="color: white; background-color: orange"> 
End    Exercise U2-5
</p>

Solution:

In [ ]:
#!/usr/bin/env python
import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
from move_robot import MoveKobuki

class LineFollower(object):
    
    def __init__(self):
        
        self.bridge_object = CvBridge()
        self.image_sub = rospy.Subscriber('/camera/rgb/image_raw', Image, self.camera_callback)
        self.movekobuki_object = MoveKobuki()
        
    def camera_callback(self, data):
        
        try:
            #We celect rgb8 because its the OpenCV encoding by default
            cv_image = self.bridge_object.image_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)
            
        # We get image dimendions and crop the parts of the image we dont need
        # Bare in mind that because its image matrix first value is start and second value is down limit.
        # Select the limits so that  it gets the line not too close, not to far and the minimum portion possible
        # To make process faster
        height, width, channels = cv_image.shape
        
        #descentre = 160
        #row_to_watch = 20
        
        #descentre = 0
        #row_to_watch = 20
        
        descentre = 0
        row_to_watch = 200
        
        crop_img = cv_image[(height)/2 + descentre:(height)/2 + (descentre + rows_to_watch)][1:width]
        
        # Convert from RGB to HSV
        hsv = cv2.cvtColor(crop_img, cv2.COLOR_RGB2HSV)
        
        # Define the Yellow Colour in HSV
        #RGB
        #[[[222, 255, 0]]]
        #BGR
        #[[[0, 255, 222]]]
        """
        To know which color to track in HSV, Put in BGR. Use ColorZilla to get the color registered by the camera
        >>> yellow = np.unit8[[[B, G,R]]]
        >>> hsv_yellow = cv2.cvtColor(yellow, cv2.COLOR_BGR2HSV)
        >>> print(hsv_yellow)
        [[[34 255 255]]]        
        """
        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([50, 255, 255])
        
        lower_yellow = np.array([0, 55, 55])
        upper_yellow = np.array([255, 255, 255])
        
        lower_yellow = np.array([33, 254, 254])
        upper_yellow = np.array([36, 255, 255])
        
        #Blue Star, RGB = [3, 70, 255] --> HSV[111, 252, 225]
        # Create a Filter for Blue star
        lower_yellow = np.array([90, 249, 200])
        upper_yellow = np.array([131, 255, 255])
        
        #Threshold the HSV image to get only yellow colors
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        # Calculate centroid of the blob of binary image using ImageMoments
        m = cv2.moments(mask, False)
        try:
            cx,cy = m['m10']/m['m00'], m['m01']/m['m00']
        except ZeroDivisionError:
            cx, cy = height/2, width/2
            
        #Bitwise-And mask and original image
        res =  cv2.bitwise_and(crop_img, crop_img, mask = mask)
        
        contours,_= cv2.findColours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1)
        rospy.loginfo("Number of centroids==>"+str(len(contours)))
        centres = []
        for i in range(len(contours)):
            moments = cv2.moments(contours[i])
            try:
                centres.append((int(moments['m10']/moments['m00']), int(moments['m01']/moments['m00'])))
                cv2.circle(res, centres[-1], 10, (0, 255, 0), -1)
            except ZeroDivisionError:
                pass
            
        rospy.loginfo(str(centres))
        #Select the right centroid
        #[(542, 39), (136, 46)], (x, y)
        most_right_centroid_index = 0
        index = 0
        max_x_value = 0
        for candidate in centres:
            #Retrieve the cx value
            cx = candidate[0]
            #Get the Cx more to the right
            if cx >= max_xvalue:
                max_xvalue = cx
                most_right_centroid_index = index
            index +=1
            
        try:
            cx = centres[most_right_centroid_index][0]
            cy = centres[most_right_centroid_index][1]
            rospy.logwarn("Winner =="+str(cx)+","+str(cy)+"")
        except:
            cx , cy = height/2, width/2
                               
        
        # Draw the centroid in the resulted image
        # cv2.circle(img, center, radius, color[. thickness[, lineType[, shift]]])
        cv2.circle(res, (int(cx), int(cy)), 10, (0,0,255), -1)
        
        cv2.imshow("Original", cv_image)
        #cv2.imshow("HSV", hsv)
        #cv2.imshow("MASK", mask)
        cv2.imshow("RES", res)
        
        cv2.waitKey(1)
        
        error_x = cx - width / 2;
        twist_object = Twist()
        twist_object.linear.x = 0.2;
        twist_object.angular.z = -error_x / 100;
        rospy.loginfo("ANGULAR VALUE SENT===>" + str(twist_object.angular.z))
        # Make it start turning
        self.movekobuki_object.move_robot(twist_object)
        
    def clean_up(self):
        self.movekobuki_object.clean_class()
        cv2.destoryAllWindows()
        
def main():
    rospy.init_node('line_following_node', anonymous=True)
        
    line_follower_object = LineFollower()
        
    rate = rospy.Rate(5)
    ctrl_c = False
    def shutdownhook():
        #works better than the rospy.is_shut_down()
        line_follower_object.clean_up()
        rospy.loginfo("shutdown time!")
        ctrl_c =  True
            
    rospy.on_shutdown(shutdownhook)
        
    while not ctrl_c:
        rate.sleep()
            
if __name__=='__main__':
    main()

As you can see its the excat same code, just that it uses only the multiple centroids and it selects the one that has the higher value of X, and therefore going always to the right. You can see also that a red circle is drawn in the centroid selected to be able to see it visually.

### PID controller with perception

Controlling a robot means moving the robot based on the info provided by some sensor data. Everything you work with perception for robots you will have this coupling. So you better start learning a little bit of control so you can take the most of your robot perception.

One way of controlling a bit better the movement of the robot and make it smoother is to apply a PID controller to the control values. Luckily for us there is a [**PID ROS package**](http://wiki.ros.org/pid) that makes using PIDs much easier.

First, you have to get used to this new tool, so here is an example for testing how the PID Package works

<p style="color: white; background-color: green"> 
    pid_test.launch
</p>

This first launch launches all the need elements for PID ROS to work and also you own PID test script "pid_control.py". <br> It launches the following:
* pkg="pid" type="control". This starts the PID control system with the values given. PID values, and also some upper and lower limits where the signal value can't exceed. It also sets the windup limit and a range in the loop frequency. This loop frequency depends on how precise you want the control to be and depends on how fast the system changes.
* pid_control.py: The one that published the values desired, and the state of the syste. It makes the function of the real system or simulated system.
* rqt_plot: Will plot the value of the PID, so the */setpoint*, the*/state* and the */control_effort* in the Graphical Tools space.
* rqt_reconfigure: This will allow you to change in execution time the PID values to see the effects immediatelly in the Graphical Tools space.

<p style="color: white; background-color: green"> 
    End pid_test.launch
</p>

<p style="color: white; background-color: green"> 
    pid_control.py
</p>

This python script works in this way:
* You set a desired value to reach, publishing in the "/setpoint" topic. In this case is "0.0"
* You also publish the "Real" state of your system in "/state" topic, in this case its a sine wave that is not affected in reality by the control. But this will make the PID system work to try and move the sine wave to the desired value 0.0
* You susbcribe to the "/control_effort", to see what values the controller is issuing to try and make the state value move to the setpoint, 0.0 in this case. It should be more or less an inverted sine wave to the state value.

In [ ]:
#!/usr/bin/env python
import rospy
from std_msgs.msg import Float64
from random import randint
from math import sin

class PID(object):
    
    def __init__(self):
        
        self._setpoint_hub = rospy.Publisher("/setpoint", Float64, queue_size=1)
        self._state_pub = rospy.Publisher("/state", Float64, queue_size=1)
        self._control_effort_sub = rospy.Subscriber('/control_effort', Float64. self.control_effort_callback)
        self._control_effort_value = Float64()
        
    def control_effort_callback(self, data):
        self._control_effort_value = data.data
        
    def setpoint_update(self, value):
        value_object = Float64()
        value_object.data = value
        self._setpoint_hub.publish(value_object)
        
    def state_update(self, value):
        value_object = Float64()
        value_object.data = value
        self._state_hub.publish(value_object)
        
    def get_control_effort(self):
        return self._control_effort_value.data
    
def sinus_test():
    rospy.init_node('sinus_pid_node', anonymous=True)
    
    pid_object = PID()
    
    rate = rospy.Rate(10.0)
    ctrl_c = False
    def shutdownhook():
        rospy.loginfo("shutdown time!")
        ctrl_c =  True
        
    rospy.on_shutdown(shutdownhook)
    setPoint_value = 0.0
    pid_object.setpoint_update(value=setPoint_value)
    
    i = 0
    
    while not ctrl_c:
        state_value = sin(i)
        pid_object.state_update(value=state_value)
        effort_value = pid_object.get_control_effort()
        #print("state_value ==>"+str(state_value))
        #print("effort_value ==>"+str(effort_value))
        rate.sleep()
        i +=0.1
        
def step_test():
    rospy.init_node('step_pid_node', anonymous=True)
    
    pid_object = PID()
    
    rate = rospy.Rate(10.0)
    ctrl_c =  False
    def shutdownhook():
        rospy.loginfo("shutdown time!")
        ctrl_c =  True
        
    rospy.on_shutdown(shutdownhook)
    setPoint_value = 0.0
    pid_object.setpoint_update(value=setPoint_value)
    
    i = 0
    state_value = 1.0
    
    while not ctrl_c:

        pid_object.state_update(value=state_value)
        effort_value = pid_object.get_control_effort()
        #print("state_value ==>"+str(state_value))
        #print("effort_value ==>"+str(effort_value))
        rate.sleep()
        i +=0.1
        if i > 30:
            state_value *= -1
            i = 0
            
if __main__=='__main__':
    #step_test()
    sinus_test()
        

You should see sth similar to this in the Graphical Tools Tab:

<img src="Images/C18.png" width="500">

What you are seeing is the step_test, where a step value of 1 and -1 is alternated. The Red line is the */set_point*. Is the value you set. And the Blue line is the effort, the signal issued by the PID to counteract that signal. Because this is dummy test, and there is no real physical system, the effort has no effect whatsoever. This is why the effort stays at an inverse value of the signal. In Real system it would affect the state and therefore the effort would diminish until the "set_point" and the "state" had the same value.
<br>
Also note that because you have set a upper_limit of 2 and lower_limit of -2, the effort values can't exceed those values. This can be seen when the Kd is increased to 1.0.

If you dont know what is going on here, please have a look at this pratical ***PID_Guide***.

<p style="color: white; background-color: green"> 
    End pid_control.py
</p>

<p style="color: white; background-color: orange"> 
    Exercise U2-6 
</p>

* Execute the code given and see the responce depending on the PID values given.
* See how increasing the Kd the values go faster but have higher peak value.
* See how setting the upper_limit and lower_limit affect also the behaviour of the effort value.

<p style="color: white; background-color: orange"> 
    END Exercise U2-6  
</p>

<p style="color: white; background-color: orange"> 
    Exercise U2-7 
</p>

Let's apply this knowledge inot the robot system.
* Create a launch file for your new follow_line python script that uses the PID.
* This launch will not only launch your follow_line_script but also the PID package for the control.
* If you want you also can plot the values with rqt_plot and start the reconfigure for tweeking the PID values.

Objective:
* Make the Robot follow smoothly the lines, no big oscilations but fast response. It has to be precise in its movements.

<p style="color: white; background-color: orange"> 
    END Exercise U2-7  
</p>

Solution:

<p style="color: white; background-color: green"> 
    pid_movement.launch
</p>

<p style="color: white; background-color: green"> 
    END pid_movement.launch
</p>

And this is the main launch:

<p style="color: white; background-color: green"> 
    follow_line_with_pid.launch 
</p>

And this is a way of moving the robot with PID. In this case no multiple centrods is searches:

<p style="color: white; background-color: green"> 
follow_line_step_pid.py
</p>

In [ ]:
#!/usr/bin/env python
import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
from move_robot import MoveKobuki
from pid_control import PID

class LineFollower(object):
    
    def __init__(self):
        rospy.lagwarn("Init line Follower")
        self.bridge_object = CvBridge()
        self.image_sub = rospy.Subscriber('/camera/rgb/image_raw', Image, self.camera_callback)
        self.movekobuki_object = MoveKobuki()
        self.pid_object = PID()
        
    def camera_callback(self, data):
        
        try:
            #We celect rgb8 because its the OpenCV encoding by default
            cv_image = self.bridge_object.image_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)
            
        # We get image dimendions and crop the parts of the image we dont need
        # Bare in mind that because its image matrix first value is start and second value is down limit.
        # Select the limits so that  it gets the line not too close, not to far and the minimum portion possible
        # To make process faster
        height, width, channels = cv_image.shape
        
        #descentre = 160
        #row_to_watch = 20
        
        #descentre = 0
        #row_to_watch = 20
        
        descentre = 0
        row_to_watch = 200
        
        crop_img = cv_image[(height)/2 + descentre:(height)/2 + (descentre + rows_to_watch)][1:width]
        
        # Convert from RGB to HSV
        hsv = cv2.cvtColor(crop_img, cv2.COLOR_RGB2HSV)
        
        # Define the Yellow Colour in HSV
        #RGB
        #[[[222, 255, 0]]]
        #BGR
        #[[[0, 255, 222]]]
        """
        To know which color to track in HSV, Put in BGR. Use ColorZilla to get the color registered by the camera
        >>> yellow = np.unit8[[[B, G,R]]]
        >>> hsv_yellow = cv2.cvtColor(yellow, cv2.COLOR_BGR2HSV)
        >>> print(hsv_yellow)
        [[[34 255 255]]]        
        """
        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([50, 255, 255])
        
        
        #Threshold the HSV image to get only yellow colors
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        # Calculate centroid of the blob of binary image using ImageMoments
        m = cv2.moments(mask, False)
        try:
            cx,cy = m['m10']/m['m00'], m['m01']/m['m00']
        except ZeroDivisionError:
            cx, cy = height/2, width/2
            
        #Bitwise-And mask and original image
        res =  cv2.bitwise_and(crop_img, crop_img, mask = mask)
                               
        
        # Draw the centroid in the resulted image
        # cv2.circle(img, center, radius, color[. thickness[, lineType[, shift]]])
        cv2.circle(res, (int(cx), int(cy)), 10, (0,0,255), -1)
        
        #cv2.imshow("Original", cv_image)
        #cv2.imshow("HSV", hsv)
        #cv2.imshow("MASK", mask)
        cv2.imshow("RES", res)
        
        cv2.waitKey(1)
        
        #Move the Robot, center it in the middle of the width 640 => 320:
        setPoint_value = width / 2
        self.pid_object.setpoint_update(value=setPoint_value)
        
        twist_object = Twist()
        twist_object.linear.x = 0.2;
        
        #Make it start turning
        self.pid_object.state_update(value=cx)
        effort_value = self.pid_object.get_control_effort()
        # We divide the effort to map it to the normal values for angular speed in the turtlebot
        rospy.lagwarn("Set Value=="+str(setPoint_value))
        rospy.lagwarn("State Value=="+str(cx))
        rospy.lagwarn("Effort Value=="+str(effort_value))
        angular_effort_value = effort_value / 200.0
         
        twist_object.angular.z = angular_effort_value;
        rospy.loginfo("Twist===>" + str(twist_object.angular.z))
        # Make it start turning
        self.movekobuki_object.move_robot(twist_object)
        
    def clean_up(self):
        self.movekobuki_object.clean_class()
        cv2.destoryAllWindows()
        
def main():
    rospy.init_node('line_following_node', anonymous=True)
        
    line_follower_object = LineFollower()
        
    rate = rospy.Rate(5)
    ctrl_c = False
    def shutdownhook():
        #works better than the rospy.is_shut_down()
        line_follower_object.clean_up()
        rospy.loginfo("shutdown time!")
        ctrl_c =  True
            
    rospy.on_shutdown(shutdownhook)
        
    while not ctrl_c:
        rate.sleep()
            
if __name__=='__main__':
    main()

In escence you are setting the "setPoint", which is always to try to have the center of the line in the middle of the image.
<br>
Then the sate, which in this case is the centroid of the blob, is published. 
<br>
And fianlly the effort given by the control PID running, is retried, converted to a sensible value then the twist is published.

<p style="color: white; background-color: orange"> 
    Exercise Extra U2-8
</p>

Create a definitive script that can follow the correct path (loop path) with the PID control and the multiple centroids info.

<p style="color: white; background-color: orange"> 
    End Exercise Extra U2-8
</p>

<p style="color: white; background-color: orange"> 
    Exercise Extra U2-9
</p>

Create a new following script that when published on a topic called "objective" the colour to follow will:
* If yellow will follow the loop path
* If red will follow the loop path until it reaches to the red star path
* If green will follow the loop path until it reaches to the green star path
* If blue will follow the loop path until it reaches to the blue star path
* If none, it will stop

<p style="color: white; background-color: orange"> 
    End Exercise Extra U2-9
</p>

## **Congratulations!**

[**OpenCV**](https://docs.opencv.org/master/d9/df8/tutorial_root.html) 

<p style="color: white; background-color: green"> 
    END line_follower_basics.py 
</p>

<img src="Images/C1.png" width="380">